In [3]:
import pandas as pd

In [4]:
import os

In [87]:
from keras.layers import Input, Embedding, Flatten, Dense
from keras.models import Model

2023-04-20 20:27:43.686558: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [223]:
from keras.models import Sequential

In [228]:
import matplotlib.pyplot as plt
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses

In [5]:
# 读取训练集的正面评论
train_pos_dir = "/Users/vanord/Desktop/alphabet/week4/IMDB/aclImdb/train/pos/"
train_pos_files = [os.path.join(train_pos_dir, f) for f in os.listdir(train_pos_dir) if f.endswith('.txt')]
pos_train = pd.concat([pd.read_csv(f, header=None, names=['review'],delimiter='\t',encoding='utf-8') for f in train_pos_files], ignore_index=True)

# 读取训练集的负面评论
train_neg_dir = "/Users/vanord/Desktop/alphabet/week4/IMDB/aclImdb/train/neg/"
train_neg_files = [os.path.join(train_neg_dir, f) for f in os.listdir(train_neg_dir) if f.endswith('.txt')]
neg_train = pd.concat([pd.read_csv(f, header=None, names=['review'],delimiter='\t',encoding='utf-8') for f in train_neg_files], ignore_index=True)

# 读取测试集的正面评论
test_pos_dir = "/Users/vanord/Desktop/alphabet/week4/IMDB/aclImdb/test/pos/"
test_pos_files = [os.path.join(test_pos_dir, f) for f in os.listdir(test_pos_dir) if f.endswith('.txt')]
pos_test = pd.concat([pd.read_csv(f, header=None, names=['review'],delimiter='\t',encoding='utf-8') for f in test_pos_files], ignore_index=True)

# 读取测试集的负面评论
test_neg_dir = "/Users/vanord/Desktop/alphabet/week4/IMDB/aclImdb/test/neg/"
test_neg_files = [os.path.join(test_neg_dir, f) for f in os.listdir(test_neg_dir) if f.endswith('.txt')]
neg_test = pd.concat([pd.read_csv(f, header=None, names=['review'],delimiter='\t',encoding='utf-8') for f in test_neg_files], ignore_index=True)

In [6]:
# 将训练集和测试集的正面评论和负面评论合并成两个数据集
train = pd.concat([pos_train, neg_train], axis=0)
test = pd.concat([pos_test, neg_test], axis=0)

# 添加标签，1表示正面评论，0表示负面评论
train['label'] = [1]*len(pos_train) + [0]*len(neg_train)
test['label'] = [1]*len(pos_test) + [0]*len(neg_test)

In [7]:
# 打印数据集信息
print(train['label'].value_counts())
print(test['label'].value_counts())

1    12500
0    12500
Name: label, dtype: int64
1    12500
0    12500
Name: label, dtype: int64


In [114]:
df = pd.read_csv('/Users/vanord/Desktop/alphabet/week4/IMDB/name.basics.tsv.gz', delimiter='\t', header=0, dtype={'nconst':str, 'primaryName':str, 'birthYear':str, 'deathYear':str, 'primaryProfession':str, 'knownForTitles':str})

In [10]:
train.head(3)

,review,label
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1


In [11]:
test_movie_id_score_pos=[]

In [12]:
for dirpath, dirnames, filenames in os.walk('/Users/vanord/Desktop/alphabet/week4/IMDB/aclImdb/test/pos/'):
    for filename in filenames:
        # 提取文件名
        file_name = os.path.splitext(filename)[0]
        # 拆分文件名
        parts = file_name.split('_')
        # 打印拆分结果
        test_movie_id_score_pos.append(parts)

In [13]:
df_test_movie_id_score_pos=pd.DataFrame(test_movie_id_score_pos,columns=['id','score'])

In [14]:
df_test_movie_id_score_pos['label_1']=1

In [15]:
df_test_movie_id_score_pos.head(3)

,id,score,label_1
0,4715,9,1
1,1930,9,1
2,3205,9,1


In [16]:
test_movie_id_score_neg=[]

In [17]:
for dirpath, dirnames, filenames in os.walk('/Users/vanord/Desktop/alphabet/week4/IMDB/aclImdb/test/neg/'):
    for filename in filenames:
        # 提取文件名
        file_name = os.path.splitext(filename)[0]
        # 拆分文件名
        parts = file_name.split('_')
        # 打印拆分结果
        test_movie_id_score_neg.append(parts)

In [18]:
df_test_movie_id_score_neg=pd.DataFrame(test_movie_id_score_neg,columns=['id','score'])

In [19]:
df_test_movie_id_score_neg['label_1']=0

In [20]:
df_test_movie_id_score_neg.head(3)

,id,score,label_1
0,1821,4,0
1,9487,1,0
2,4604,4,0


In [21]:
df_test_movie_id_score=pd.concat([df_test_movie_id_score_pos, df_test_movie_id_score_neg], axis=0)

In [22]:
df_test_movie_id_score.label_1.value_counts()

1    12500
0    12500
Name: label_1, dtype: int64

In [23]:
test=pd.concat([test,df_test_movie_id_score], axis=1)

In [24]:
test=test.drop(columns='label_1')

In [25]:
test.head(3)

,review,label,id,score
0,"Based on an actual story, John Boorman shows t...",1,4715,9
1,This is a gem. As a Film Four production - the...,1,1930,9
2,"I really like this show. It has drama, romance...",1,3205,9


In [26]:
train_movie_id_score_pos=[]

In [27]:
for dirpath, dirnames, filenames in os.walk('/Users/vanord/Desktop/alphabet/week4/IMDB/aclImdb/train/pos/'):
    for filename in filenames:
        # 提取文件名
        file_name = os.path.splitext(filename)[0]
        # 拆分文件名
        parts = file_name.split('_')
        # 打印拆分结果
        train_movie_id_score_pos.append(parts)

In [28]:
df_train_movie_id_score_pos=pd.DataFrame(train_movie_id_score_pos,columns=['id','score'])

In [29]:
df_train_movie_id_score_pos.head(3)

,id,score
0,4715,9
1,12390,8
2,8329,7


In [30]:
train_movie_id_score_neg=[]

In [31]:
for dirpath, dirnames, filenames in os.walk('/Users/vanord/Desktop/alphabet/week4/IMDB/aclImdb/train/neg/'):
    for filename in filenames:
        # 提取文件名
        file_name = os.path.splitext(filename)[0]
        # 拆分文件名
        parts = file_name.split('_')
        # 打印拆分结果
        train_movie_id_score_neg.append(parts)

In [32]:
df_train_movie_id_score_neg=pd.DataFrame(train_movie_id_score_neg,columns=['id','score'])

In [33]:
df_train_movie_id_score_neg['label_1']=0

In [34]:
df_train_movie_id_score_pos['label_1']=1

In [35]:
df_test_movie_id_score_neg.shape

(12500, 3)

In [36]:
df_test_movie_id_score_pos.shape

(12500, 3)

In [37]:
df_train_movie_id_score=pd.concat([df_train_movie_id_score_pos, df_train_movie_id_score_neg], axis=0)

In [38]:
train=pd.concat([train,df_train_movie_id_score], axis=1)

In [39]:
train=train.drop(columns='label_1')

In [40]:
train.head(10)

,review,label,id,score
0,For a movie that gets no respect there sure ar...,1,4715,9
1,Bizarre horror movie filled with famous faces ...,1,12390,8
2,"A solid, if unremarkable film. Matthau, as Ein...",1,8329,7
3,It's a strange feeling to sit alone in a theat...,1,9063,8
4,"You probably all already know this by now, but...",1,3092,10
5,I saw the movie with two grown children. Altho...,1,9865,8
6,You're using the IMDb.<br /><br />You've given...,1,6639,10
7,This was a good film with a powerful message o...,1,10460,10
8,"Made after QUARTET was, TRIO continued the qua...",1,10331,10
9,"For a mature man, to admit that he shed a tear...",1,11606,10


In [115]:
df['knownForTitles']=df['knownForTitles'].apply(lambda x: x.split(','))

In [116]:
df['primaryProfession']=df['primaryProfession'].apply(lambda x: str(x).split(','))

In [122]:
df.head(3)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"[soundtrack, actor, miscellaneous]","[tt0050419, tt0031983, tt0053137, tt0072308]"
1,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0117057, tt0038355, tt0071877, tt0037382]"
2,nm0000003,Brigitte Bardot,1934,\N,"[actress, soundtrack, music_department]","[tt0056404, tt0049189, tt0054452, tt0057345]"


In [123]:
df[df['deathYear']=='\\N']['birthYear'].min()

'0012'

In [124]:
df['deathYear_ab']=df['deathYear'].apply(lambda x: 1 if x.startswith('0') else 0)

In [125]:
df['birthYear_ab']=df['birthYear'].apply(lambda x: 1 if x.startswith('0') else 0)

In [126]:
df=df[(df['deathYear_ab']==0) & (df['birthYear_ab']==0)]

In [127]:
df=df[df['birthYear']!='\\N']

In [128]:
df.shape

(552216, 8)

In [129]:
df['birthYear'].value_counts().index.max()

'2022'

In [130]:
df.head(10)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,deathYear_ab,birthYear_ab
0,nm0000001,Fred Astaire,1899,1987,"[soundtrack, actor, miscellaneous]","[tt0050419, tt0031983, tt0053137, tt0072308]",0,0
1,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0117057, tt0038355, tt0071877, tt0037382]",0,0
2,nm0000003,Brigitte Bardot,1934,\N,"[actress, soundtrack, music_department]","[tt0056404, tt0049189, tt0054452, tt0057345]",0,0
3,nm0000004,John Belushi,1949,1982,"[actor, soundtrack, writer]","[tt0080455, tt0072562, tt0078723, tt0077975]",0,0
4,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0050986, tt0060827, tt0050976, tt0083922]",0,0
5,nm0000006,Ingrid Bergman,1915,1982,"[actress, soundtrack, producer]","[tt0077711, tt0038109, tt0036855, tt0034583]",0,0
6,nm0000007,Humphrey Bogart,1899,1957,"[actor, soundtrack, producer]","[tt0034583, tt0037382, tt0042593, tt0043265]",0,0
7,nm0000008,Marlon Brando,1924,2004,"[actor, soundtrack, director]","[tt0047296, tt0068646, tt0070849, tt0078788]",0,0
8,nm0000009,Richard Burton,1925,1984,"[actor, soundtrack, producer]","[tt0057877, tt0059749, tt0087803, tt0061184]",0,0
9,nm0000010,James Cagney,1899,1986,"[actor, soundtrack, director]","[tt0031867, tt0035575, tt0042041, tt0029870]",0,0


In [131]:
df[df['deathYear']=='\\N']

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,deathYear_ab,birthYear_ab
2,nm0000003,Brigitte Bardot,1934,\N,"[actress, soundtrack, music_department]","[tt0056404, tt0049189, tt0054452, tt0057345]",0,0
46,nm0000047,Sophia Loren,1934,\N,"[actress, soundtrack]","[tt0076085, tt0058335, tt0054749, tt0060121]",0,0
78,nm0000079,Raquel Welch,1940,\N,"[actress, soundtrack, producer]","[tt0072281, tt0070291, tt0061653, tt0066115]",0,0
83,nm0000084,Gong Li,1965,\N,"[actress, producer]","[tt0473444, tt0101640, tt0397535, tt0430357]",0,0
84,nm0000085,Henner Hofmann,1950,\N,"[cinematographer, producer, camera_department]","[tt0097523, tt0113482, tt1825758, tt0097738]",0,0
...,...,...,...,...,...,...,...,...
11671564,nm9992812,Daisy,1994,\N,[actress],[tt8281190],0,0
11671668,nm9992946,Christoph Blaschke,1993,\N,"[miscellaneous, assistant_director, camera_dep...","[tt17321230, tt9738336, tt10172548, tt11650404]",0,0
11671692,nm9992971,Shamica Anderson,1988,\N,"[editor, cinematographer, producer]","[tt12788416, tt13702196, tt11910974, tt17888710]",0,0
11671893,nm9993199,Denis Nurulin,1997,\N,[actor],"[tt8721954, tt12351166, tt16689786, tt13756156]",0,0


In [132]:
df=df.replace('\\N','2023')

In [133]:
df['age']=df['deathYear'].astype(int)-df['birthYear'].astype(int)

In [134]:
df=df.drop(columns=['deathYear_ab','birthYear_ab'])

In [135]:
df.head(10)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,age
0,nm0000001,Fred Astaire,1899,1987,"[soundtrack, actor, miscellaneous]","[tt0050419, tt0031983, tt0053137, tt0072308]",88
1,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0117057, tt0038355, tt0071877, tt0037382]",90
2,nm0000003,Brigitte Bardot,1934,2023,"[actress, soundtrack, music_department]","[tt0056404, tt0049189, tt0054452, tt0057345]",89
3,nm0000004,John Belushi,1949,1982,"[actor, soundtrack, writer]","[tt0080455, tt0072562, tt0078723, tt0077975]",33
4,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0050986, tt0060827, tt0050976, tt0083922]",89
5,nm0000006,Ingrid Bergman,1915,1982,"[actress, soundtrack, producer]","[tt0077711, tt0038109, tt0036855, tt0034583]",67
6,nm0000007,Humphrey Bogart,1899,1957,"[actor, soundtrack, producer]","[tt0034583, tt0037382, tt0042593, tt0043265]",58
7,nm0000008,Marlon Brando,1924,2004,"[actor, soundtrack, director]","[tt0047296, tt0068646, tt0070849, tt0078788]",80
8,nm0000009,Richard Burton,1925,1984,"[actor, soundtrack, producer]","[tt0057877, tt0059749, tt0087803, tt0061184]",59
9,nm0000010,James Cagney,1899,1986,"[actor, soundtrack, director]","[tt0031867, tt0035575, tt0042041, tt0029870]",87


In [136]:
df.primaryProfession.value_counts().index[:4]

Index([['actor'], ['actress'], ['nan'], ['writer']], dtype='object')

In [137]:
df['primaryProfession_ab']=df['primaryProfession'].apply(lambda x: 1 if 'nan' in x else 0)

In [138]:
df=df[df['primaryProfession_ab']==0]

In [139]:
df=df.drop(columns=['primaryProfession_ab'])

In [140]:
df['knownForTitles_ab']=df['knownForTitles'].apply(lambda x: 1 if '\\N' in x else 0)

In [141]:
df=df[df['knownForTitles_ab']==0]

In [142]:
df=df.drop(columns='knownForTitles_ab')

In [143]:
assert sum(df['nconst'].str.startswith('nm'))==df.shape[0]

In [145]:
df.shape[0]

478522

In [148]:
occupation_list=[]

In [150]:
range(df.shape[0])

range(0, 478522)

In [157]:
df=df.reset_index()

In [158]:
for i in range(478522):
    for j in df['primaryProfession'][i]:
        if j in occupation_list:
            break
        else:
            occupation_list.append(j)

In [205]:
occupation_list

['soundtrack',
 'actor',
 'miscellaneous',
 'actress',
 'music_department',
 'writer',
 'director',
 'producer',
 'make_up_department',
 'composer',
 'assistant_director',
 'camera_department',
 'cinematographer',
 'editor',
 'art_director',
 'stunts',
 'production_designer',
 'costume_designer',
 'editorial_department',
 'sound_department',
 'visual_effects',
 'production_manager',
 'location_management',
 'costume_department',
 'special_effects',
 'set_decorator',
 'art_department',
 'casting_department',
 'casting_director',
 'script_department',
 'animation_department',
 'transportation_department',
 'manager',
 'talent_agent',
 'executive',
 'publicist',
 'legal',
 'assistant']

In [206]:
len(occupation_list)

38

In [207]:
occupation_dict={}

In [208]:
occupation_enu=enumerate(occupation_list,start=0)

In [209]:
for count, value in occupation_enu:
    occupation_dict.update({count:value})

In [210]:
occupation_dict

{0: 'soundtrack',
 1: 'actor',
 2: 'miscellaneous',
 3: 'actress',
 4: 'music_department',
 5: 'writer',
 6: 'director',
 7: 'producer',
 8: 'make_up_department',
 9: 'composer',
 10: 'assistant_director',
 11: 'camera_department',
 12: 'cinematographer',
 13: 'editor',
 14: 'art_director',
 15: 'stunts',
 16: 'production_designer',
 17: 'costume_designer',
 18: 'editorial_department',
 19: 'sound_department',
 20: 'visual_effects',
 21: 'production_manager',
 22: 'location_management',
 23: 'costume_department',
 24: 'special_effects',
 25: 'set_decorator',
 26: 'art_department',
 27: 'casting_department',
 28: 'casting_director',
 29: 'script_department',
 30: 'animation_department',
 31: 'transportation_department',
 32: 'manager',
 33: 'talent_agent',
 34: 'executive',
 35: 'publicist',
 36: 'legal',
 37: 'assistant'}

In [211]:
new_dict={v:k for k,v in occupation_dict.items()}

In [212]:
new_dict

{'soundtrack': 0,
 'actor': 1,
 'miscellaneous': 2,
 'actress': 3,
 'music_department': 4,
 'writer': 5,
 'director': 6,
 'producer': 7,
 'make_up_department': 8,
 'composer': 9,
 'assistant_director': 10,
 'camera_department': 11,
 'cinematographer': 12,
 'editor': 13,
 'art_director': 14,
 'stunts': 15,
 'production_designer': 16,
 'costume_designer': 17,
 'editorial_department': 18,
 'sound_department': 19,
 'visual_effects': 20,
 'production_manager': 21,
 'location_management': 22,
 'costume_department': 23,
 'special_effects': 24,
 'set_decorator': 25,
 'art_department': 26,
 'casting_department': 27,
 'casting_director': 28,
 'script_department': 29,
 'animation_department': 30,
 'transportation_department': 31,
 'manager': 32,
 'talent_agent': 33,
 'executive': 34,
 'publicist': 35,
 'legal': 36,
 'assistant': 37}

In [215]:
df['occupation']=df['primaryProfession'].apply(lambda x: [new_dict.get(i) for i in x])

In [217]:
df.head(20)

,index,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,age,occupation
0,0,nm0000001,Fred Astaire,1899,1987,"[soundtrack, actor, miscellaneous]","[tt0050419, tt0031983, tt0053137, tt0072308]",88,"[0, 1, 2]"
1,1,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0117057, tt0038355, tt0071877, tt0037382]",90,"[3, 0]"
2,2,nm0000003,Brigitte Bardot,1934,2023,"[actress, soundtrack, music_department]","[tt0056404, tt0049189, tt0054452, tt0057345]",89,"[3, 0, 4]"
3,3,nm0000004,John Belushi,1949,1982,"[actor, soundtrack, writer]","[tt0080455, tt0072562, tt0078723, tt0077975]",33,"[1, 0, 5]"
4,4,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0050986, tt0060827, tt0050976, tt0083922]",89,"[5, 6, 1]"
5,5,nm0000006,Ingrid Bergman,1915,1982,"[actress, soundtrack, producer]","[tt0077711, tt0038109, tt0036855, tt0034583]",67,"[3, 0, 7]"
6,6,nm0000007,Humphrey Bogart,1899,1957,"[actor, soundtrack, producer]","[tt0034583, tt0037382, tt0042593, tt0043265]",58,"[1, 0, 7]"
7,7,nm0000008,Marlon Brando,1924,2004,"[actor, soundtrack, director]","[tt0047296, tt0068646, tt0070849, tt0078788]",80,"[1, 0, 6]"
8,8,nm0000009,Richard Burton,1925,1984,"[actor, soundtrack, producer]","[tt0057877, tt0059749, tt0087803, tt0061184]",59,"[1, 0, 7]"
9,9,nm0000010,James Cagney,1899,1986,"[actor, soundtrack, director]","[tt0031867, tt0035575, tt0042041, tt0029870]",87,"[1, 0, 6]"


In [86]:
usr_emb = Embedding(num_embeddings=df.shape[0],
                embedding_dim=32,
                sparse=False)

In [88]:
df['nconst_id'] = pd.factorize(df['nconst'])[0]

In [100]:
df['age_bin'] = pd.cut(df['age'], bins=[0, 18, 25, 35, 45, 55, 65, 75, np.inf], labels=False)

In [103]:
df.head(10)

,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,age,nconst_id,age_bin
0,Fred Astaire,1899,1987,"[soundtrack, actor, miscellaneous]","[tt0050419, tt0031983, tt0053137, tt0072308]",88,0,7.0
1,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0117057, tt0038355, tt0071877, tt0037382]",90,1,7.0
2,Brigitte Bardot,1934,2023,"[actress, soundtrack, music_department]","[tt0056404, tt0049189, tt0054452, tt0057345]",89,2,7.0
3,John Belushi,1949,1982,"[actor, soundtrack, writer]","[tt0080455, tt0072562, tt0078723, tt0077975]",33,3,2.0
4,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0050986, tt0060827, tt0050976, tt0083922]",89,4,7.0
5,Ingrid Bergman,1915,1982,"[actress, soundtrack, producer]","[tt0077711, tt0038109, tt0036855, tt0034583]",67,5,6.0
6,Humphrey Bogart,1899,1957,"[actor, soundtrack, producer]","[tt0034583, tt0037382, tt0042593, tt0043265]",58,6,5.0
7,Marlon Brando,1924,2004,"[actor, soundtrack, director]","[tt0047296, tt0068646, tt0070849, tt0078788]",80,7,7.0
8,Richard Burton,1925,1984,"[actor, soundtrack, producer]","[tt0057877, tt0059749, tt0087803, tt0061184]",59,8,5.0
9,James Cagney,1899,1986,"[actor, soundtrack, director]","[tt0031867, tt0035575, tt0042041, tt0029870]",87,9,7.0


In [90]:
df=df.drop(columns='nconst')

In [92]:
nconst_input = Input(shape=(1,), name='nconst_input')

In [94]:
embedding_layer = Embedding(input_dim=df['nconst_id'].nunique(), output_dim=32, name='nconst_embedding')(nconst_input)

In [95]:
flatten_layer = Flatten()(embedding_layer)

In [96]:
hidden_layer = Dense(units=64, activation='relu')(flatten_layer)
output_layer = Dense(units=1, activation='sigmoid')(hidden_layer)

In [99]:
embedding_layer

<KerasTensor: shape=(None, 1, 32) dtype=float32 (created by layer 'nconst_embedding')>

In [105]:
# 定义age输入层
age_input = Input(shape=(1,), name='age_input')

# 定义嵌入层，将age_bin映射为向量表示
embedding_layer = Embedding(input_dim=df['age_bin'].nunique(), output_dim=32, name='age_embedding')(age_input)

In [106]:
# 展平嵌入向量
flatten_layer = Flatten()(embedding_layer)

# 定义全连接层和输出层
hidden_layer = Dense(units=64, activation='relu')(flatten_layer)
output_layer = Dense(units=1, activation='sigmoid')(hidden_layer)

In [218]:
occupation_input = Input(shape=(1,), name='occupation_input')
occupation_embedding = Embedding(input_dim=38, output_dim=32, input_length=1, name='occupation_embedding')(occupation_input)
flatten_layer = Flatten()(occupation_embedding)

In [219]:
hidden_layer = Dense(units=64, activation='relu')(flatten_layer)
output_layer = Dense(units=1, activation='sigmoid')(hidden_layer)

In [225]:
# 构建电影ID编码字典
movie_dict = {}
for index, row in df.iterrows():
    titles = row['knownForTitles']
    for title in titles:
        if title not in movie_dict:
            movie_dict[title] = len(movie_dict)


# 构建embedding模型
max_len = df['knownForTitles'].apply(len).max()  # 最长电影ID序列长度
embedding_size = 16
model = Sequential()
model.add(Embedding(len(movie_dict), embedding_size, input_length=max_len))
model.add(Flatten())



In [226]:
train.head(3)

,review,label,id,score
0,For a movie that gets no respect there sure ar...,1,4715,9
1,Bizarre horror movie filled with famous faces ...,1,12390,8
2,"A solid, if unremarkable film. Matthau, as Ein...",1,8329,7


In [227]:
test.head(3)

,review,label,id,score
0,"Based on an actual story, John Boorman shows t...",1,4715,9
1,This is a gem. As a Film Four production - the...,1,1930,9
2,"I really like this show. It has drama, romance...",1,3205,9


In [229]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [230]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [231]:
vectorize_layer.adapt(train['review'])

In [232]:
def vectorize_text(text):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text)

In [233]:
train_data=vectorize_text(train['review'])

In [234]:
test_data=vectorize_text(test['review'])